# NPM3D Panoptic Segmentation — embeddings + eval (ETH pipeline)

**Repo**: fork of `prs-eth/PanopticSegForLargeScalePointCloud`.

Ce notebook Colab installe l'environnement, récupère NPM3D (avec labels d'instances),
lance l'entraînement/éval avec les configs des auteurs, **extrait les embeddings 5D** avant le clustering,
et calcule les métriques officielles (F1 / PQ / etc.).

⚙️ GPU requis (Colab Pro conseillé).

## 1) Vérifier le GPU

In [ ]:
import platform

def version_tuple(ver: str):
    main = ver.split("+", 1)[0]
    parts = main.split(".")
    nums = []
    for part in parts:
        if part.isdigit():
            nums.append(int(part))
        else:
            digits = ""
            for ch in part:
                if ch.isdigit():
                    digits += ch
                else:
                    break
            if digits:
                nums.append(int(digits))
                break
    while len(nums) < 3:
        nums.append(0)
    return tuple(nums[:3])

try:
    import torch
except Exception as exc:
    torch = None
    print("Torch non importable pour le moment:", exc)
else:
    print("Torch version:", torch.__version__)
    vt = version_tuple(torch.__version__)
    if vt < (2, 8, 0):
        print("⚠️ PyTorch >= 2.8.0 est recommandé pour ce notebook.")
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))

!nvidia-smi || true


## 2) Dépendances système (OpenBLAS pour MinkowskiEngine)

In [ ]:

%%bash
set -euxo pipefail
sudo apt-get update -y
sudo apt-get install -y build-essential git cmake libopenblas-dev libomp-dev
# Optional but useful for large builds
sudo apt-get install -y ninja-build



## 3) PyTorch 2.8.0 (cu126) + PyG compatibles + MinkowskiEngine
Ce notebook part du principe que PyTorch 2.8.0+cu126 est déjà présent dans l'environnement (par ex. `pip install torch==2.8.0+cu126` ou `pip install torch==2.8.0+cu128`).
La cellule suivante aligne les bibliothèques PyG compatibles, installe les dépendances Python nécessaires et (re)compile MinkowskiEngine avec OpenBLAS.

* Tu peux surcharger l'URL des roues PyG via la variable d'environnement `PYG_INDEX_URL` si tu disposes d'un miroir interne.
* Si l'accès au dépôt officiel `https://data.pyg.org` est filtré (erreurs 403 via proxy, etc.), exporte `ALLOW_PYG_SOURCE_BUILD=1` avant d'exécuter la cellule pour autoriser une compilation locale depuis les sources (`torch-scatter`, `torch-sparse`, `torch-cluster`). Cette étape peut durer plus de 20 minutes sur une machine fraîche.
* Pense à configurer `HTTP_PROXY`/`HTTPS_PROXY`/`NO_PROXY` selon ton infrastructure réseau pour que `pip` atteigne les dépôts distants.


In [ ]:

    import os
    import shlex
    import subprocess
    import sys
    import urllib.error
    import urllib.request
    from pathlib import Path

    def run(cmd, **kwargs):
        if isinstance(cmd, (list, tuple)):
            printable = " ".join(shlex.quote(str(c)) for c in cmd)
        else:
            printable = str(cmd)
        print(f"+ {printable}")
        subprocess.check_call(cmd, **kwargs)

    PYTHON = sys.executable
    run([PYTHON, "-m", "pip", "install", "--upgrade", "pip", "wheel", "setuptools"])

    try:
        import torch
    except Exception as exc:
        raise SystemExit("Veuillez installer PyTorch 2.8.0+cu126 (ou compatible) avant d'exécuter ce notebook.") from exc


    def version_tuple(ver: str):
        main = ver.split("+", 1)[0]
        parts = main.split(".")
        nums = []
        for part in parts:
            if part.isdigit():
                nums.append(int(part))
            else:
                digits = ""
                for ch in part:
                    if ch.isdigit():
                        digits += ch
                    else:
                        break
                if digits:
                    nums.append(int(digits))
                    break
        while len(nums) < 3:
            nums.append(0)
        return tuple(nums[:3])


    if version_tuple(torch.__version__) < (2, 8, 0):
        raise SystemExit(f"PyTorch >= 2.8.0 requis, version détectée: {torch.__version__}")

    print(f"PyTorch {torch.__version__} (CUDA {torch.version.cuda or 'CPU'})")

    base_ver = torch.__version__.split("+", 1)[0]
    build_suffix = torch.__version__.split("+", 1)[1] if "+" in torch.__version__ else ""
    if build_suffix.startswith("cu"):
        cu_tag = build_suffix
    else:
        cuda_version = torch.version.cuda.replace(".", "") if torch.version.cuda else ""
        cu_tag = f"cu{cuda_version}" if cuda_version else "cpu"

    custom_index = os.environ.get("PYG_INDEX_URL")
    if custom_index:
        pyg_index = custom_index
        print(f"Index PyG forcé via $PYG_INDEX_URL: {pyg_index}")
    else:
        if cu_tag == "cpu":
            pyg_index = f"https://data.pyg.org/whl/torch-{base_ver}.html"
        else:
            pyg_index = f"https://data.pyg.org/whl/torch-{base_ver}+{cu_tag}.html"
        print(f"Index PyG calculé: {pyg_index}")


    def pyg_index_accessible(url: str) -> bool:
        if not url:
            return False
        if url.startswith("http"):
            try:
                with urllib.request.urlopen(url, timeout=10) as resp:
                    print(f"[pyg] accès {url} -> HTTP {resp.status}")
                    return 200 <= resp.status < 300
            except urllib.error.HTTPError as exc:
                print(f"[pyg] accès {url} refusé: HTTP {exc.code} ({exc.reason}).")
                return False
            except urllib.error.URLError as exc:
                print(f"[pyg] accès {url} impossible: {exc}.")
                return False
        else:
            p = Path(url)
            ok = p.exists()
            print(f"[pyg] accès local {url}: {'ok' if ok else 'introuvable'}")
            return ok
        return False


    run([
        PYTHON,
        "-m",
        "pip",
        "install",
        "--no-cache-dir",
        "--upgrade",
        "hydra-core==1.3.2",
        "omegaconf==2.3.0",
        "plyfile==0.9",
        "scipy==1.11.4",
        "hdbscan==0.8.33",
        "pandas==2.2.2",
        "numba==0.59.1",
        "joblib==1.3.2",
        "tqdm",
        "pyyaml",
        "packaging",
    ])

    pyg_packages = [
        "torch-scatter==2.1.2",
        "torch-sparse==0.6.18",
        "torch-cluster==1.6.3",
        "torch-geometric==2.6.1",
    ]

    wheels_available = pyg_index_accessible(pyg_index)
    allow_source_build = os.environ.get("ALLOW_PYG_SOURCE_BUILD", "").strip().lower() in {"1", "true", "yes"}
    if not wheels_available and not allow_source_build:
        raise SystemExit(
            "Impossible de télécharger les roues PyG depuis data.pyg.org.
"
            "Configure HTTP(S)_PROXY/NO_PROXY ou fixe $PYG_INDEX_URL vers un miroir accessible.
"
            "Sinon exporte ALLOW_PYG_SOURCE_BUILD=1 pour compiler les bibliothèques PyG depuis les sources (cela peut prendre"
            " 20+ minutes)."
        )

    pyg_cmd = [
        PYTHON,
        "-m",
        "pip",
        "install",
        "--no-cache-dir",
        "--upgrade",
    ]
    if wheels_available:
        pyg_cmd.extend(["--extra-index-url", pyg_index])
    else:
        print("[pyg] Compilation depuis les sources demandée (ALLOW_PYG_SOURCE_BUILD=1).")
        pyg_cmd.extend(["--no-binary", "torch-scatter,torch-sparse,torch-cluster"])
    pyg_cmd.extend(pyg_packages)
    run(pyg_cmd)

    repo_dir = Path("MinkowskiEngine")
    if repo_dir.exists():
        run(["git", "-C", str(repo_dir), "pull", "--ff-only"])
    else:
        run(["git", "clone", "--depth", "1", "https://github.com/NVIDIA/MinkowskiEngine.git", str(repo_dir)])

    env = os.environ.copy()
    env.setdefault("MAX_JOBS", str(os.cpu_count() or 1))
    run([
        PYTHON,
        "setup.py",
        "install",
        "--blas_include_dirs=/usr/include",
        "--blas=openblas",
    ], cwd=str(repo_dir), env=env)

    import MinkowskiEngine as ME
    print("MinkowskiEngine:", ME.__version__)


## 4) Cloner votre fork et préparer l'arborescence

In [ ]:

%%bash
set -euxo pipefail
REPO_URL="https://github.com/Ludwig-H/PanopticSegForLargeScalePointCloud"
if [ ! -d PanopticSegForLargeScalePointCloud ]; then
  git clone --depth=1 "$REPO_URL"
fi
cd PanopticSegForLargeScalePointCloud
# Créer dossiers de données selon README
mkdir -p data/npm3dfused/raw
mkdir -p outputs
echo "Repo ready."


## 5) Télécharger NPM3D (avec labels d’instances)
Source officielle Zenodo `10.5281/zenodo.8118986`. Les fichiers .ply sont posés dans `data/npm3dfused/raw/`.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud/data/npm3dfused/raw

base="https://zenodo.org/records/8118986/files"

files=(
  "Paris_train.ply"
  "Paris_val.ply"
  "Paris_test.ply"
  "Lille1_1_train.ply"
  "Lille1_1_val.ply"
  "Lille1_1_test.ply"
  "Lille1_2_train.ply"
  "Lille1_2_val.ply"
  "Lille1_2_test.ply"
  "Lille2_train.ply"
  "Lille2_val.ply"
  "Lille2_test.ply"
)

for f in "${files[@]}"; do
  if [ ! -f "$f" ]; then
    echo "Downloading $f"
    wget -q --show-progress "${base}/${f}?download=1" -O "$f"
  else
    echo "Already have $f"
  fi
done


## 6) (Optionnel) Entraînement NPM3D — même config que le papier
Les auteurs donnent en exemple Setting **IV** pour `area1`:

```bash
python train.py task=panoptic \
  data=panoptic/npm3d-sparseconv_grid_012_R_16_cylinder_area1 \
  models=panoptic/area4_ablation_3heads_5 model_name=PointGroup-PAPER \
  training=7_area1 job_name=A1_S7
```
Tu peux boucler sur `area{1..4}` pour faire le 4-fold.
Met `WANDB_MODE=offline` pour éviter la synchro en ligne.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud

export WANDB_MODE=offline
# Dé-commenter pour lancer un entraînement complet pour area1 (coûteux).
# python train.py task=panoptic \
#   data=panoptic/npm3d-sparseconv_grid_012_R_16_cylinder_area1 \
#   models=panoptic/area4_ablation_3heads_5 model_name=PointGroup-PAPER \
#   training=7_area1 job_name=A1_S7
echo "Entraînement désactivé par défaut (trop long en Colab)."


## 7) Extraire les **embeddings 5D** avant clustering
On attache un **forward hook** sur la branche d'embedding du modèle pendant l'éval.
Le script ci-dessous:
- charge la config Hydra d'éval, le checkpoint, et le dataloader test (mêmes scènes),
- capture les embeddings par point et les enregistre en `.npz` par bloc.

Il essaie d'abord des clés de sortie usuelles (`embedding`, `embeddings`, `embedding_5d`),
puis cherche un module dont le nom contient `emb` pour y accrocher un hook si besoin.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud

cat > tools_extract_embeddings.py << 'PY'
import os, sys, glob, json, re, pathlib, numpy as np, torch
from pathlib import Path
from omegaconf import OmegaConf
sys.path.append(str(Path.cwd()))
# Import TP3D local copy
sys.path.append(str(Path.cwd() / "torch_points3d"))
from torch_points3d.metrics.panoptic_tracker import PanopticTracker  # noqa: F401 (forces TP3D import)
from train import hydra_main as train_hydra_main  # to reuse config utils if exposed
from eval import hydra_main as eval_hydra_main    # type: ignore

# Small utility to load hydra config from conf/eval.yaml
def load_eval_cfg():
    cfg = OmegaConf.load("conf/eval.yaml")
    return cfg

def find_embed_module(model):
    candidates = []
    for name, m in model.named_modules():
        if "emb" in name.lower():
            candidates.append(name)
    return candidates

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def dump_npz(out_dir, tag, data_dict):
    ensure_dir(out_dir)
    np.savez_compressed(os.path.join(out_dir, f"{tag}.npz"), **data_dict)

def main():
    import hydra
    from omegaconf import DictConfig
    # Run eval hydra to build model + dataloaders then intercept forward
    @hydra.main(config_path="conf", config_name="eval", version_base=None)
    def _run(cfg: DictConfig):
        # Build model & loaders the same way eval.py does
        # We call the eval hydra entry but override the evaluation loop to attach hooks.
        from eval import build_model_and_loaders  # we expect eval.py to expose helpers; fallback otherwise
        try:
            model, loaders, device, test_split = build_model_and_loaders(cfg)
        except Exception as e:
            print("Falling back to importing torch_points3d style builders:", e)
            # Fallback: try to import internal builders if authors changed names
            raise

        model.eval()
        model = model.to(device)

        # Try to discover an embedding head via outputs or module names
        caught = {"batch_indices": [], "embeddings": []}
        example_name = None

        # Soft hook mechanism
        chosen_mod = None
        for name, m in model.named_modules():
            if re.search(r"(emb|embed|embedding)", name, re.I):
                chosen_mod = m
                chosen_name = name
                break
        hook_handle = None
        if chosen_mod is not None:
            def _hook(module, inp, out):
                # out shape [N, D]
                try:
                    E = out.detach().cpu().float().numpy()
                    caught["embeddings"].append(E)
                except Exception:
                    pass
            hook_handle = chosen_mod.register_forward_hook(lambda m,i,o: _hook(m,i,o))
            print(f"[extract] Hooked embedding module: {chosen_name}", flush=True)
        else:
            print("[extract] No obvious embedding module found; will try dict outputs.", flush=True)

        out_root = Path("outputs/embeddings")
        ensure_dir(out_root)

        # Iterate over test loader only
        test_loader = loaders.get(test_split, None) or loaders.get("test", None)
        if test_loader is None:
            raise RuntimeError("Test loader not found in loaders keys:", loaders.keys())

        with torch.no_grad():
            for ib, batch in enumerate(test_loader):
                batch = batch.to(device)
                out = model(batch)
                # If model returns dict with embeddings
                if isinstance(out, dict):
                    for k in ["embedding", "embeddings", "embedding_5d", "emb_5d", "panoptic_embeddings"]:
                        if k in out and isinstance(out[k], torch.Tensor):
                            E = out[k].detach().cpu().float().numpy()
                            caught["embeddings"].append(E)
                            break

                # Gather point indices to map back if available
                if hasattr(batch, "idx"):
                    idx = batch.idx.detach().cpu().numpy()
                elif hasattr(batch, "ptr"):
                    idx = batch.ptr.detach().cpu().numpy()
                else:
                    # fallback: monotonic indices
                    n = out["semantic"].shape[0] if isinstance(out, dict) and "semantic" in out else 0
                    idx = np.arange(n, dtype=np.int64)

                # Flatten and dump per-batch
                if len(caught["embeddings"]) == 0:
                    print(f"[extract] WARNING: no embeddings captured for batch {ib}.")
                    continue
                E = caught["embeddings"][-1]
                tag = f"batch_{ib:05d}"
                dump_npz(out_root, tag, {"indices": idx, "embeddings": E})
                print(f"[extract] Saved embeddings for {tag} -> {E.shape}", flush=True)

        if hook_handle is not None:
            hook_handle.remove()

    _run()

if __name__ == "__main__":
    main()
PY

echo "Script tools_extract_embeddings.py created."


## 8) Exécuter l'évaluation et l’extraction d’embeddings
Le script d’éval officiel produit les dossiers `viz_for_test_*`. Ensuite, on extrait les embeddings en suivant **les mêmes splits/scènes**.

**Remarque:** pense à mettre le chemin du checkpoint (`CKPT_PATH`) si tu n’entraînes pas dans cette session.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud

# Exemple: on veut évaluer Area 1 (Paris en test) en Setting IV (comme dans README)
# Si tu as entraîné ici, adapte CKPT_PATH en conséquence; sinon, place un checkpoint existant à l'endroit indiqué.
CKPT_PATH="${CKPT_PATH:-/content/drive/MyDrive/panoptic_runs/A1_S7/checkpoints/best.pth}" || true

# Adapter conf/eval.yaml avant de lancer eval.py (les auteurs l'indiquent dans le README).
# Comme on ne connaît pas tes chemins exacts, on te laisse la main:
echo "Ouvre et adapte conf/eval.yaml si nécessaire (paths, split, ckpt)."
sed -n '1,160p' conf/eval.yaml || true

# Lancer l'éval (si eval.py dépend entièrement de conf/eval.yaml)
# python eval.py

# Ou, si Hydra accepte des overrides, tu peux donner des précisions en CLI (à adapter si besoin) :
# python eval.py task=panoptic data=panoptic/npm3d-sparseconv_grid_012_R_16_cylinder_area1 \
#   models=panoptic/area4_ablation_3heads_5 model_name=PointGroup-PAPER training=7_area1 \
#   +eval.ckpt_path="$CKPT_PATH"

echo "Extraction des embeddings (utilise conf/eval.yaml et le même loader test)"
python tools_extract_embeddings.py || true

echo "Pour calculer les métriques officielles NPM3D :"
echo "python evaluation_stats_NPM3D.py"


## 9) Où sont les embeddings et comment brancher ton clusterer
Les `.npz` sont écrits sous `outputs/embeddings/` avec deux arrays:
- `embeddings`: matrice `[N, 5]` (D=5 attendu) de l’embedding discriminant par point du batch
- `indices`: indices des points pour reconstituer l’ordre si besoin

### Exemple: brancher un clusterer perso qui prend soit `X` soit une matrice de distances
Le snippet ci-dessous charge tous les `.npz`, concatène, normalise à la volée, applique un clusterer,
puis sauvegarde un `instance_id` par point dans un `.npy`. À toi de remplacer `my_clusterer()`.

In [ ]:

import os, glob, numpy as np
from sklearn.preprocessing import StandardScaler

EMB_DIR = "PanopticSegForLargeScalePointCloud/outputs/embeddings"
OUT_PATH = "PanopticSegForLargeScalePointCloud/outputs/instance_labels.npy"

def load_all_embeddings_npz(emb_dir):
    files = sorted(glob.glob(os.path.join(emb_dir, "*.npz")))
    Xs, Is = [], []
    for f in files:
        d = np.load(f)
        Xs.append(d["embeddings"])
        Is.append(d["indices"])
    if not Xs:
        raise RuntimeError("Aucun fichier d'embedding trouvé dans %s" % emb_dir)
    X = np.concatenate(Xs, axis=0)
    I = np.concatenate(Is, axis=0)
    return X, I

def my_clusterer(X):
    # TODO: remplace par ton algo. Ici, on met un stub trivial à 1 cluster.
    # X: [N, d]
    N = X.shape[0]
    labels = np.zeros(N, dtype=np.int32)
    return labels

X, I = load_all_embeddings_npz(EMB_DIR)
Xn = StandardScaler().fit_transform(X)
labels = my_clusterer(Xn)
np.save(OUT_PATH, labels)
print("Saved instance labels to:", OUT_PATH, " shape:", labels.shape)


## 10) (Facultatif) Remonter tes clusters en PLY et relancer l’éval officielle
Si tu veux remplacer les instances du pipeline par les tiennes, convertis `instance_labels.npy` en `.ply` attendu par `evaluation_stats_NPM3D.py`.
Le format attendu par l’issue #19 indique des champs `pre_sem_label`, `gt_sem_label` et les IDs d’instance prédits/GT. Tu peux partir des sorties de `viz_for_test_valid_proposals` et ne remplacer que la colonne d’instances prédictives.

In [ ]:

# Exemple de squelette: à adapter selon les champs déjà présents dans vos PLY de sortie
# from plyfile import PlyData, PlyElement
# import numpy as np
# sem_ply = PlyData.read("PanopticSegForLargeScalePointCloud/viz_for_test_valid_proposals/instance.ply")
# labels = np.load("PanopticSegForLargeScalePointCloud/outputs/instance_labels.npy")
# # Construire une nouvelle structure en remplaçant le champ d'instance prédite puis sauvegarder.
# PlyData([ ... ]).write("PanopticSegForLargeScalePointCloud/outputs/instance_custom.ply")
print("Voir issue #19 pour les champs consultés par evaluation_stats_NPM3D.py")


## 11) Lancer l’évaluation finale (scripts officiels)
Les auteurs fournissent `evaluation_stats_NPM3D.py` qui charge les sorties d’éval et calcule F1, PQ, etc.

In [ ]:

%%bash
set -euxo pipefail
cd PanopticSegForLargeScalePointCloud
# Pour NPM3D
python evaluation_stats_NPM3D.py || true
# Pour FOR-instance (si utilisé)
# python evaluation_stats_FOR.py


## Notes pratiques
- Les chemins et le checkpoint à utiliser pour l’éval se règlent dans `conf/eval.yaml` ou via des overrides Hydra. Le README fournit les commandes et les noms des configs pour NPM3D, **Setting IV** notamment.
- Le dataset NPM3D avec **labels d’instance** se télécharge depuis Zenodo (12 fichiers .ply). On les place sous `data/npm3dfused/raw/` comme indiqué dans le README.
- Si l’extraction d’embeddings ne capture rien du premier coup, imprime la liste des `model.named_modules()` et choisis un module contenant `emb` pour le hook.